In [1]:
from math import log
import pandas as pd
import numpy as np
import random
print(1)

data = pd.read_csv("Bidding.csv")[['Bidding_Ratio','Early_Bidding','Class']]
data
print(inforgain(data, 'Bidding_Ratio', 'Class'))
print(inforgain(data, 'Early_Bidding', 'Class'))

data = pd.read_csv("Bidding.csv")[['Bidding_Ratio','Early_Bidding','Class']]
data
choose_best_col(data,"Class")

# 第四版回归决策树

In [7]:
class ID3Tree:
    class Node: 
        def __init__(self, name, type_node):
            self.name = name
            self.left = None
            self.right = None
            self.type = type_node
            self.key = None
    
    def __init__(self):
        None

    
    # label需要是DataFrame
    def fit(self, data, label, largest_layer = 5):
        self.data = pd.concat([data, label], axis=1)
        self.columns = list(self.data.columns)
        self.label = label.columns[0]
        self.largest_layer = largest_layer
        self.root = self.buildtree(self.data, 1)
        # self.print_tree(self.root)
        
    # 打印树
    def print_tree(self, root):
        if root.type == "leaf":
            print("leaf节点,预测值为{name}".format(name = root.name))
            return
        else: 
            print("root节点,划分属性为{name},划分值为{key}".format(name = root.name, key = root.key))
        self.print_tree(root.left)
        self.print_tree(root.right)


    

    # ele为标签列表
    def entropy(self, ele):    
        probs = [ele.count(i)/len(ele) for i in set(ele)]    #计算ele集合中k类样本分布
        entropy = -sum([prob*log(prob, 2) for prob in probs])  #计算 
        return entropy

    # data需要包含标签列
    def inforgain(self, df, column):
        entropy_D = self.entropy(df[self.label].tolist())
        # 得到划分点的集合
        valuelist = list(set(data[column].tolist()))
        valuelist.sort()
        Ta = []
        for i in range(len(valuelist)-1):
            Ta.append((valuelist[i] + valuelist[i+1])/2)
        # 对不同划分方式依次计算其熵
        best_gain = -999
        for split_value in Ta:
            D1 = df[df[column] <= split_value]
            D2 = df[df[column] > split_value]
            entropy_D1 = self.entropy(D1[self.label].tolist())
            entropy_D2 = self.entropy(D2[self.label].tolist())
            entropy_DA = len(D1)/len(df) * entropy_D1 + len(D2)/len(df) * entropy_D2
            gain = entropy_D - entropy_DA
            if gain > best_gain:
                best_gain = gain
                best_split_value = split_value
        # 返回信息增益, 最佳分割的值
        return best_gain , best_split_value

    def choose_best_col(self, df):
        columns = [i for i in list(df.columns) if i != self.label ]
        best_gain, best_col_key, best_col= -99, -99, None
        for col in columns:
            col_gain, col_key = self.inforgain(df, col)
            if col_gain > best_gain:
                best_gain = col_gain
                best_col_key = col_key
                best_col = col
        return best_col, best_col_key
    
    # 递归调用函数
    # input_data需要包含标签列, k表示当前节点所在层数
    def buildtree(self, input_data, k):
        # 如果大于最大层数
        if k >= self.largest_layer:
            leafnode = self.Node(np.mean(input_data[self.label]), "leaf") #叶节点
            return leafnode

        # 若df中只有一种类型则该节点变为叶节点
        if len(input_data[self.label].unique()) == 1:
            leafnode = self.Node(input_data[self.label].unique()[0], "leaf") #叶节点     
            return leafnode
            
        # 当df中所有属性的值都是一样
        if (input_data.drop(columns = self.label).duplicated().sum() == (len(input_data)-1)):
            leafnode = self.Node(np.mean(input_data[self.label]), "leaf") #叶节点
            return leafnode

        best_col, best_col_key = self.choose_best_col(input_data)
        node = self.Node(best_col, "root")
        node.key = best_col_key
        node.left = self.buildtree(input_data[input_data[best_col] <= best_col_key], k+1)
        node.right = self.buildtree(input_data[input_data[best_col] > best_col_key], k+1)
        return node

    def predictSingle(self, sample, node):
        if node.type == "leaf":
            return node.name
        value = sample[node.name].values[0]
        if value <= node.key:
            nextNode = node.left
            return self.predictSingle(sample, nextNode)
        else:
            nextNode = node.right
            return self.predictSingle(sample, nextNode)

    def predictDf(self, data):
        prediction = []
        for i in range(len(data)) :
            sample = data[i:(i+1)]
            prediction.append(self.predictSingle(sample, self.root))
        return prediction

## 测试

In [8]:
tree = ID3Tree()
data = pd.read_csv("Bidding.csv")[['Bidding_Ratio','Early_Bidding']][1:300]
label = pd.read_csv("Bidding.csv")[["Class"]][1:300]
test = pd.read_csv("Bidding.csv")[['Bidding_Ratio','Early_Bidding']][301:311]
test_label = pd.read_csv("Bidding.csv")[["Class"]][301:311]
tree.fit(data,label, 6)
print(tree.predictDf(test))
print(test_label.values.squeeze())

root节点,划分属性为Bidding_Ratio,划分值为0.1297335205
root节点,划分属性为Early_Bidding,划分值为0.863606096
leaf节点,预测值为0
root节点,划分属性为Early_Bidding,划分值为0.868650463
leaf节点,预测值为1
root节点,划分属性为Bidding_Ratio,划分值为0.065942029
leaf节点,预测值为0
root节点,划分属性为Bidding_Ratio,划分值为0.07504690450000001
leaf节点,预测值为1.0
leaf节点,预测值为0.0
root节点,划分属性为Early_Bidding,划分值为0.0017048609999999998
leaf节点,预测值为0
root节点,划分属性为Bidding_Ratio,划分值为0.3397435895
root节点,划分属性为Early_Bidding,划分值为0.0137843915
leaf节点,预测值为0
root节点,划分属性为Bidding_Ratio,划分值为0.13397129149999998
leaf节点,预测值为1.0
leaf节点,预测值为0.2857142857142857
root节点,划分属性为Bidding_Ratio,划分值为0.4083333335
root节点,划分属性为Bidding_Ratio,划分值为0.3660714285
leaf节点,预测值为1.0
leaf节点,预测值为0.0
leaf节点,预测值为1
[0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 0 0]


## 交叉验证

In [9]:
# y需要是数据框
def CVR(reg,X,y,Z=10,seed=1010):
    np.random.seed(seed)
    # 从np.repeat(np.arange(Z), np.ceil(len(y)/Z))数组中抽取，长度为len(y)的样本，不可重复
    m = np.random.choice(np.repeat(np.arange(Z), np.ceil(len(y)/Z)), len(y), replace=False)
    p0 = np.ones(len(y))
    pcv= np.ones(len(y))
    p0 = reg.fit(X,y)
    p0 = reg.predictDf(X) #模型预测值
    NMSE0 = np.sum((p0-y.values.squeeze())**2)/np.sum((np.mean(y.values.squeeze())-y.values.squeeze())**2)
    for i in range(Z):
        pcv[m == i] = reg.fit(X[m != i], y[m != i])
        pcv[m == i] = reg.predictDf(X[m == i]) # pcv[m==i]的长度是len(y)/Z
        NMSEcv=np.sum((pcv-y.values.squeeze())**2)/np.sum((np.mean(y.values.squeeze())-y.values.squeeze())**2)
    return pd.DataFrame({"Data":["Train","Test"],"NMSE": [NMSE0, NMSEcv]})
    
a = ID3Tree()
CVR(a, data, label)


root节点,划分属性为Bidding_Ratio,划分值为0.1297335205
root节点,划分属性为Early_Bidding,划分值为0.863606096
leaf节点,预测值为0
root节点,划分属性为Early_Bidding,划分值为0.868650463
leaf节点,预测值为1
root节点,划分属性为Bidding_Ratio,划分值为0.065942029
leaf节点,预测值为0.0
leaf节点,预测值为0.07142857142857142
root节点,划分属性为Early_Bidding,划分值为0.0017048609999999998
leaf节点,预测值为0
root节点,划分属性为Bidding_Ratio,划分值为0.3397435895
root节点,划分属性为Early_Bidding,划分值为0.0137843915
leaf节点,预测值为0.0
leaf节点,预测值为0.3076923076923077
root节点,划分属性为Bidding_Ratio,划分值为0.4083333335
leaf节点,预测值为0.6666666666666666
leaf节点,预测值为1.0
root节点,划分属性为Bidding_Ratio,划分值为0.1297335205
root节点,划分属性为Early_Bidding,划分值为0.863606096
leaf节点,预测值为0
root节点,划分属性为Early_Bidding,划分值为0.868650463
leaf节点,预测值为1
root节点,划分属性为Bidding_Ratio,划分值为0.065942029
leaf节点,预测值为0.0
leaf节点,预测值为0.08333333333333333
root节点,划分属性为Bidding_Ratio,划分值为0.4083333335
root节点,划分属性为Early_Bidding,划分值为0.0568650795
leaf节点,预测值为0
root节点,划分属性为Bidding_Ratio,划分值为0.13397129149999998
leaf节点,预测值为1.0
leaf节点,预测值为0.29411764705882354
root节点,划分属性为Early_Bidding,划分值为6.48e-05


,Data,NMSE
0,Train,0.534552
1,Test,0.774758


# 第二版决策树

class ID3Tree:    
    # 定义多分支节点：属性为节点名，下属节点用字典表示(字典的键表示当前节点的属性值，值是按照该属性值划分后得到的下一节点)
    class Node:        
        def __init__(self, name):
            self.name = name
            self.connections = {}    
        # 建立连接，增加一个叫label为键(name的属性值)，node为值的键值对
        def connect(self, label, node):
            self.connections[label] = node    
    
    # label指的是列名
    def __init__(self, data, label):
        self.columns = list(data.columns)
        self.data = data
        self.label = label
        self.root = self.Node("Root")    
    
    # 打印树
    def print_tree(self, node, tabs):
        print(tabs + node.name)        
        for connection, child_node in node.connections.items():
            print(tabs + "\t" +"(" + str(connection) + ")")#tabs + "\t" + 
            self.print_tree(child_node, tabs + "\t\t")   

    # 得到分支数据集，根据在全数据框col(属性)中出现的数值，每个数值(属性值)得到一个数据集
    def split_dataframe(self, split_data, col):    
        unique_values = self.data[col].unique()    #得到col列中有哪些数值
        # 创建分类字典
        result_dict = {elem : pd.DataFrame() for elem in unique_values}   
        # 根据列值分割数据框,如果没有分割值则为空
        for key in split_data[col].unique():
            result_dict[key] = split_data[:][split_data[col] == key].drop(columns=[col])
        # result_dict中键是col列中的数值，值是一个df(是原数据col列的值是键的所有行组成的，不包括col列)
        return result_dict


    # 找到给定一个df,找到最佳的列(属性).label指定df中那一列是标签
    # 将会返回：在当前给定的样本集合下,最大信息增益，最佳划分属性(列名)，划分之后的子集合字典
    def choose_best_col(self, df, label): 
        # 当前集合信息熵
        entropy_D = entropy(df[label].tolist())    
        # 得到列名，除了标签那一列, 可能会返回[]
        cols = [col for col in df.columns if col not in [label]]    
        # 初始化:最大信息增益，最佳列(属性)和最佳拆分字典
        max_value, best_col = -999, None
        max_splited = None

        # 根据不同的列拆分数据
        for col in cols:
            splited_set = self.split_dataframe(df, col)
            entropy_DA = 0

            # 每次取出键(属性值)和值(df包括有lable列) 
            for subset_col, subset in splited_set.items():
                if subset.empty == True:
                    continue
                # 计算熵
                entropy_Di = entropy(subset[label].tolist())            
                # 计算出如果以当前(列)属性为划分,则信息熵为
                entropy_DA += len(subset)/len(df) * entropy_Di

            # 计算出如果以当前(列)属性为划分,则信息增益为
            info_gain = entropy_D - entropy_DA

            if info_gain > max_value:
                max_value, best_col = info_gain, col
                max_splited = splited_set  
        # 对每一个col都检测一遍之后就可以得知哪一个是最佳划分属性了
        # 得到最佳划分属性名，以及以该属性
        return best_col, max_splited



    # 由根节点构建树构建树
    def construct_tree(self):
        self.root.connect("", self.construct(self.root, self.data, self.columns))
        self.print_tree(self.root, "-")
    
    # 构建树的递归函数
    # 参数包括父节点，input_data(包括标签列)，剩余属性值(包括标签列)
    def construct(self, parent_node, input_data, columns): 

        # 若df中只有一种类型则该节点变为叶节点
        if len(input_data[self.label].unique()) == 1:
            leafnode = self.Node(input_data[self.label].unique()[0]) #叶节点           
            return leafnode

        if (columns == [self.label]) or (input_data.drop(columns = self.label).duplicated().sum() == (len(input_data)-1)):
            input_data_label_list = input_data[self.label].tolist()
            leafnode = self.Node(max(input_data_label_list, key = input_data_label_list.count)) #叶节点
            return leafnode


        # 找到当前最合适的列
        best_col, max_splited = self.choose_best_col(input_data[columns], self.label) 
        # 得到最优划分属性
        node = self.Node(best_col)                                  #根节点
        new_columns = [col for col in columns if col != best_col]  #剩余属性 

        # 递归构造决策树
        for splited_value, splited_data in max_splited.items():
            if splited_data.empty == True :
                input_data_label_list = input_data[self.label].tolist()
                leafnode = self.Node(max(input_data_label_list, key = input_data_label_list.count)) #叶节点
                node.connect(splited_value, leafnode)
                continue
            node.connect(splited_value, self.construct(node, splited_data, new_columns))
        return node
            

    def predictSingle(self, sample, node):
        # 如果来到叶节点就返回预测值
        if node.name in (self.data[self.label].tolist()):
            return node.name
        value = sample[node.name].values[0]       #取得值
        nextNode = node.connections[value]        #得到下一个节点
        prediction = self.predictSingle(sample, nextNode)
        return prediction
    
    def predictDf(self, data):
        prediction = []
        for i in range(len(data)) :
            sample = data[i:(i+1)]
            prediction.append(self.predictSingle(sample, self.root.connections[""]))
        return prediction


    data =  [[0, 0, 0, 0, 'no'],         #数据集
            [0, 0, 0, 1, 'no'],
            [0, 1, 0, 1, 'yes'],
            [0, 1, 1, 0, 'yes'],
            [0, 0, 0, 0, 'no'],
            [1, 0, 0, 0, 'no'],
            [1, 0, 0, 1, 'no'],
            [1, 1, 1, 1, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [2, 0, 1, 2, 'yes'],
            [2, 0, 1, 1, 'yes'],
            [2, 1, 0, 1, 'yes'],
            [2, 1, 0, 2, 'yes'],
            [1, 0, 0, 2, 'no'],
            [2, 0, 0, 0, 'yes'],
            [0, 1, 0, 1, 'no'],
            [0, 1, 0, 2, 'yes'],
            [0, 1, 0, 1, 'no'],
            [0, 0, 1, 2, 'yes'],
            [2, 1, 0, 0, 'no'],
            [0, 0, 1, 1, 'yes'],
            [2, 1, 0, 2, 'no'],
            [1, 0, 0, 2, 'yes'],
            [1, 0, 1, 0, 'no']]
    data = pd.DataFrame(data, columns=['年龄', '有工作', '有自己的房子', '信贷情况', '类别'])
    decisionTree = ID3Tree(data, "类别")
    decisionTree.construct_tree()

    testdata =  [[2, 1, 0, 2],
                [1, 0, 1, 2],
                [0, 0, 0, 0],
                [1, 0, 0, 2],
                [0, 0, 1, 2],
                [0, 1, 0, 0],
                [0, 1, 0, 1],
                [2, 1, 0, 0],
                [1, 1, 0, 2],
                [2, 0, 0, 1]]
    testdata = pd.DataFrame(testdata, columns=['年龄', '有工作', '有自己的房子', '信贷情况'])
    decisionTree.predictDf(testdata)

## 第三版决策树

In [ ]:
# 计算熵，ele是标签集合[1,2,1,2,3,1,3,2,2,1,2]
def entropy(ele):    
    probs = [ele.count(i)/len(ele) for i in set(ele)]    #计算ele集合中k类样本分布
    entropy = -sum([prob*log(prob, 2) for prob in probs])  #计算 
    return entropy
    
class ID3Tree:
    # 定义多分支节点：属性为节点名，下属节点用字典表示(字典的键表示当前节点的属性值，值是按照该属性值划分后得到的下一节点)
    class Node:        
        def __init__(self, name):
            self.name = name
            self.connections = {}    
        # 建立连接，增加一个叫label为键(name的属性值)，node为值的键值对
        def connect(self, label, node):
            self.connections[label] = node    
    

    def __init__(self):
        self.root = self.Node("Root")

    
    # 打印树
    def print_tree(self, node, tabs):
        print(tabs + str(node.name))        
        for connection, child_node in node.connections.items():
            print(tabs + "\t" +"(" + str(connection) + ")")#tabs + "\t" + 
            self.print_tree(child_node, tabs + "\t\t")   

    # 得到分支数据集，根据在全数据框col(属性)中出现的数值，每个数值(属性值)得到一个数据集
    def split_dataframe(self, split_data, col):    
        unique_values = self.data[col].unique()    #得到col列中有哪些数值
        # 创建分类字典
        result_dict = {elem : pd.DataFrame() for elem in unique_values}   
        # 根据列值分割数据框,如果没有分割值则为空
        for key in split_data[col].unique():
            result_dict[key] = split_data[:][split_data[col] == key].drop(columns=[col])
        # result_dict中键是col列中的数值，值是一个df(是原数据col列的值是键的所有行组成的，不包括col列)
        return result_dict


    # 找到给定一个df,找到最佳的列(属性).label指定df中那一列是标签
    # 将会返回：在当前给定的样本集合下,最大信息增益，最佳划分属性(列名)，划分之后的子集合字典
    def choose_best_col(self, df, label): 
        # 当前集合信息熵
        entropy_D = entropy(df[label].tolist())    
        # 得到列名，除了标签那一列, 可能会返回[]
        cols = [col for col in df.columns if col not in [label]]    
        # 初始化:最大信息增益，最佳列(属性)和最佳拆分字典
        max_value, best_col = -999, None
        max_splited = None

        # 根据不同的列拆分数据
        for col in cols:
            splited_set = self.split_dataframe(df, col)
            entropy_DA = 0

            # 每次取出键(属性值)和值(df包括有lable列) 
            for subset_col, subset in splited_set.items():
                if subset.empty == True:
                    continue
                # 计算熵
                entropy_Di = entropy(subset[label].tolist())            
                # 计算出如果以当前(列)属性为划分,则信息熵为
                entropy_DA += len(subset)/len(df) * entropy_Di

            # 计算出如果以当前(列)属性为划分,则信息增益为
            info_gain = entropy_D - entropy_DA

            if info_gain > max_value:
                max_value, best_col = info_gain, col
                max_splited = splited_set  
        # 对每一个col都检测一遍之后就可以得知哪一个是最佳划分属性了
        # 得到最佳划分属性名，以及以该属性
        return best_col, max_splited



    # 由根节点构建树构建树,label必须是数据框
    def fit(self, data, label):
        self.data = pd.concat([data, label], axis=1)
        self.columns = list(self.data.columns)
        self.label = label.columns[0]
        self.root.connect("", self.construct(self.root, self.data, self.columns))
        # self.print_tree(self.root, "-")
    
    # 树的递归函数
    # 参数包括父节点，input_data(包括标签列)，剩余属性值(包括标签列)
    def construct(self, parent_node, input_data, columns): 
        # 若df中只有一种类型则该节点变为叶节点
        if len(input_data[self.label].unique()) == 1:
            leafnode = self.Node(input_data[self.label].unique()[0]) #叶节点     
            return leafnode

        if (columns == [self.label]) or (input_data.drop(columns = self.label).duplicated().sum() == (len(input_data)-1)):
            input_data_label_list = input_data[self.label].tolist()
            leafnode = self.Node(max(input_data_label_list, key = input_data_label_list.count)) #叶节点
            return leafnode


        # 找到当前最合适的列
        best_col, max_splited = self.choose_best_col(input_data[columns], self.label) 
        # 得到最优划分属性
        node = self.Node(best_col)                                  #根节点
        new_columns = [col for col in columns if col != best_col]  #剩余属性 

        # 递归构造决策树
        for splited_value, splited_data in max_splited.items():
            if splited_data.empty == True :
                input_data_label_list = input_data[self.label].tolist()
                leafnode = self.Node(max(input_data_label_list, key = input_data_label_list.count)) #叶节点
                node.connect(splited_value, leafnode)
                continue
            node.connect(splited_value, self.construct(node, splited_data, new_columns))
        return node
            

    def predictSingle(self, sample, node):
        # 如果来到叶节点就返回预测值
        if node.name in (self.data[self.label].tolist()):
            return node.name
        value = sample[node.name].values[0]       #取得值
        nextNode = node.connections[value]        #得到下一个节点
        prediction = self.predictSingle(sample, nextNode)
        return prediction
    
    def predictDf(self, data):
        prediction = []
        for i in range(len(data)) :
            sample = data[i:(i+1)]
            prediction.append(self.predictSingle(sample, self.root.connections[""]))
        return prediction


In [ ]:
a =str(3)
eval(a+">=2")

import random
[[random.randint(0,2), random.randint(0,1), random.randint(0,1), random.randint(0,2)] for i in range(10)]

## 测试

In [ ]:
data =  [[random.randint(0,2), random.randint(0,1), random.randint(0,1), random.randint(0,2)] for i in range(10)] #构造数据集
data = pd.DataFrame(data, columns=['年龄', '有工作', '有自己的房子', '信贷情况'])
print(data)
label = pd.DataFrame(np.random.choice([0,1], len(data)), columns = ["类别"])
print(label)
decisionTree = ID3Tree()
decisionTree.fit(data, label)
decisionTree.print_tree(decisionTree.root, "-")

In [ ]:
testdata =  [[2, 1, 0, 2],
            [1, 0, 1, 2],
            [0, 0, 0, 0],
            [1, 0, 0, 2],
            [0, 0, 1, 2],
            [0, 1, 0, 0],
            [0, 1, 0, 1],
            [2, 1, 0, 0],
            [1, 1, 0, 2],
            [2, 0, 0, 1]]
testdata = pd.DataFrame(testdata, columns=['年龄', '有工作', '有自己的房子', '信贷情况'])
testdata.describe()

## 交叉验证函数

In [ ]:
# y需要是数据框
def CVR(reg,X,y,Z=10,seed=1010):
    np.random.seed(seed)
    # 从np.repeat(np.arange(Z), np.ceil(len(y)/Z))数组中抽取，长度为len(y)的样本，不可重复
    m = np.random.choice(np.repeat(np.arange(Z), np.ceil(len(y)/Z)), len(y), replace=False)
    p0 = np.ones(len(y))
    pcv= np.ones(len(y))
    p0 = reg.fit(X,y)
    p0 = reg.predictDf(X) #模型预测值
    NMSE0 = np.sum((p0-y.values.squeeze())**2)/np.sum((np.mean(y.values.squeeze())-y.values.squeeze())**2)
    for i in range(Z):
        pcv[m == i] = reg.fit(X[m != i], y[m != i])
        pcv[m == i] = reg.predictDf(X[m == i]) # pcv[m==i]的长度是len(y)/Z
        NMSEcv=np.sum((pcv-y.values.squeeze())**2)/np.sum((np.mean(y.values.squeeze())-y.values.squeeze())**2)
    return pd.DataFrame({"Data":["Train","Test"],"NMSE": [NMSE0, NMSEcv]})
a = ID3Tree()
CVR(a, data, label)

In [ ]:
a